In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'C:\\Users\\kisho\\MyWork\\RedWineQuality'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str

In [5]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        create_directories([config.root_dir])
        
        model_eval_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=self.schema.TARGET_COLUMN.name
        )
        return model_eval_config
    

In [6]:
import os
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from src.utils.common import save_json
import pandas as pd
import numpy as np


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    @staticmethod
    def eval_metrics(actual, predicted):
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        return rmse, mae, r2
    
    def save_results(self):
        
        test = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        X_test = test.drop(columns = [self.config.target_column])
        y_test = test[self.config.target_column]
        
        predicted = model.predict(X_test)

        rmse, mae, r2 = ModelEvaluation.eval_metrics(y_test, predicted)
        
        scores = {
            "Root Mean Squared Error": rmse,
            "Mean Absolute Error": mae,
            "R2 Score": r2
        }
        
        save_json(path=Path(self.config.metric_file_name), data=scores)
        

In [7]:
from src.logging import logger

try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config=model_eval_config)
    model_eval.save_results()
except Exception as e:
    logger.exception(f"{e}")
    raise e

[2025-03-21 18:30:44,341: INFO: common: config\config.yaml file loaded successfully]
[2025-03-21 18:30:44,341: INFO: common: schema.yaml file loaded successfully]
[2025-03-21 18:30:44,357: INFO: common: params.yaml file loaded successfully]
Directory 'artifacts' already exists.
Directory 'artifacts/model_evaluation' already exists.
[2025-03-21 18:30:44,592: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
